In [5]:
import horton
import numpy as np

In [10]:
def get_n2_energy(bond_length, basis_set, use_CO_basis=False):
    horton.log.set_level(0)

    mol = horton.IOData()
    bond_length = float(bond_length)
    mol.coordinates = np.array([[0.0, 0.0, -bond_length/2.], [0.0, 0.0, bond_length/2.]]) * horton.angstrom
    mol.numbers = np.array([7., 7.])
    mol.pseudo_numbers = mol.numbers
    
    if use_CO_basis:
        obasis = horton.get_gobasis(mol.coordinates, np.array([7, 7]), basis_set)
    else:
        obasis = horton.get_gobasis(mol.coordinates, np.array([8, 6]), basis_set)
    
    olp = obasis.compute_overlap()
    kin = obasis.compute_kinetic()
    na = obasis.compute_nuclear_attraction(mol.coordinates, mol.pseudo_numbers)
    er = obasis.compute_electron_repulsion()
    
    orb_alpha = horton.Orbitals(obasis.nbasis)
    orb_beta = horton.Orbitals(obasis.nbasis)
    
    horton.guess_core_hamiltonian(olp, kin + na, orb_alpha, orb_beta)
    
    external = {'nn': horton.compute_nucnuc(mol.coordinates, mol.pseudo_numbers)}
    
    terms = [horton.UTwoIndexTerm(kin, 'kin'), horton.UDirectTerm(er, 'hartree'), horton.UExchangeTerm(er, 'x_hf'), horton.UTwoIndexTerm(na, 'ne')]
    ham = horton.UEffHam(terms, external)
    occ_model = horton.AufbauOccModel(7, 7)
    occ_model.assign(orb_alpha, orb_beta)
    dm_alpha = orb_alpha.to_dm()
    dm_beta = orb_beta.to_dm()
    scf_solver = horton.ODASCFSolver(1e-5, maxiter=400)
    scf_solver(ham, olp, occ_model, dm_alpha, dm_beta)
    
    return ham.compute_energy()

In [15]:
for basisset in ('STO-3G', '6-31G(d)', 'Def2-TZVP', 'cc-pvtz'):
    print (basisset, get_n2_energy(2., basisset, True) - get_n2_energy(2., basisset, False))

('STO-3G', -1.9490517906819775)
('6-31G(d)', -1.7538657427202224)
('Def2-TZVP', -0.07845902612203304)
('cc-pvtz', -0.6656797254418336)
